* 진형님 판시사항_명사_형태소_ngram 참조

* 데이터 처리시 알아야 할 7가지 


inplace=True 

memory_usage="deep"

%timeit pd.read_csv("")

pd.read_csv("", usecols=[])

.dtype

.memory_usage()

.shape[0]

.nunique()

.astype("")

astype("Sparse[float32]")

data = pd.read_csv("", usecols=[], dtype={"key": np.int16, "key2": "category"})

for chunk in pd.read_csv("", chunksize=5000):

.groupby

# 1. 데이터 로드_판례본문

저장된 데이터 가져오기

In [1]:
import pandas as pd
import re

In [2]:
import matplotlib.pyplot as plt

# 한글 깨짐 방지
plt.rcParams['font.family'] = "Malgun Gothic"

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [8]:
# 파일은 전처리 완료된 판례본문 / 판례이유 분리하여 열로 저장한 것
df = pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\df_labor_real_cleaned2.csv', encoding='utf-8')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    5016 non-null   int64 
 1   판례정보일련번호      5016 non-null   int64 
 2   사건번호          5016 non-null   object
 3   선고            5016 non-null   object
 4   판결유형          5016 non-null   object
 5   판시사항          3725 non-null   object
 6   판결요지          3054 non-null   object
 7   참조조문          3724 non-null   object
 8   참조판례          2450 non-null   object
 9   판례내용          5016 non-null   object
 10  판례내용_상단       5016 non-null   object
 11  판례내용_이유       5012 non-null   object
 12  판례내용_이유(전처리)  5016 non-null   object
 13  판례내용_제거(불용어)  5016 non-null   object
dtypes: int64(2), object(12)
memory usage: 548.8+ KB


In [19]:
#메모리 할당량 계산

df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    5016 non-null   int64 
 1   판례정보일련번호      5016 non-null   int64 
 2   사건번호          5016 non-null   object
 3   선고            5016 non-null   object
 4   판결유형          5016 non-null   object
 5   판시사항          3725 non-null   object
 6   판결요지          3054 non-null   object
 7   참조조문          3724 non-null   object
 8   참조판례          2450 non-null   object
 9   판례내용          5016 non-null   object
 10  판례내용_상단       5016 non-null   object
 11  판례내용_이유       5012 non-null   object
 12  판례내용_이유(전처리)  5016 non-null   object
 13  판례내용_제거(불용어)  5016 non-null   object
dtypes: int64(2), object(12)
memory usage: 232.4 MB


In [21]:
#csv 파일을 로드하는 런타임 계산

%timeit pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\df_labor_real_cleaned2.csv', encoding='utf-8')

2.56 s ± 34.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
# 범주형 데이터를 나타내는 열의 데이터 유형 변경
# 데이터를 나타내는 열의 데이터 유형 변경

print("레코드 수:", df_labor.shape[0])
print("고유한 열값의 수:", df_labor.판시사항.nunique())

레코드 수: 5016


NameError: name 'da_labor' is not defined

In [28]:
print(df_labor.판례내용.dtype)

object


불용어 제거하기 

In [13]:
#한국어 처리를 의한 라이브러리 사용 
# Counter: 데이터를 카운팅하여 빈도수를 계산하기 위한 모듈, 주로 단어 빈도 계산에 사용
from collections import Counter

# konlpy의 Okt: 한국어 형태소 분석기. 텍스트를 형태소 단위로 분리하거나 품사 태깅에 사용
from konlpy.tag import Okt

# CountVectorizer: 문서에서 단어 빈도 수를 기반으로 피처 벡터를 생성하는 사이킷런의 도구
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import pandas as pd
import re  # 정규 표현식 모듈
from konlpy.tag import Okt

# 불용어 파일 로드
stopwords_file_path = r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\sorted_words.txt'
stopwords_pick = ['제', '호로', '등', '것', '항', '의', '금', '검사', '판결', '거나', '유1', 
                  '상고이유', '기초', '사실', '기초사실', '사고', '피고', '원고', '피고인', 
                  '사건', '담당', '변호사', '변호인', '변론', '종결', '소외']
pattern = r'제\d+|\d호증|\d호'  # 정규 표현식 패턴

# 불용어 파일을 읽어 불용어 목록 생성
with open(stopwords_file_path, 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# 불용어 제거 함수 (리스트 상태로 처리)
def remove_stopwords(content):
    # Okt 객체를 생성하여 형태소 분석
    okt = Okt()
    
    # 형태소 분석을 통해 단어 리스트 생성
    tokens = okt.morphs(content)  # 형태소 단위로 분리
    filtered_words = [
        word for word in tokens 
        if len(word) > 1 
        and word not in stopwords 
        and not word.isdigit()  # 숫자인 경우 제외
        and word not in stopwords_pick 
        and not re.match(pattern, word)
    ]
    return ' '.join(filtered_words)  # 문자열로 반환

# 불용어 제거 수행 (판례내용 열의 내용을 처리)
df_labor['판례내용_제거(불용어)'] = df_labor['판례내용'].apply(remove_stopwords)

# 결과 확인
print(df_labor['판례내용_제거(불용어)'])

판례내용 길이분포 확인

# 2. 데이터 분석

### 형태소 분포

### n-gram

In [ ]:
N-gram

In [5]:
import pandas as pd
from collections import Counter
from konlpy.tag import Okt
import matplotlib.pyplot as plt

### 2-1 표준할당

In [10]:
df_labor = df.copy()

In [12]:
df_labor['판례내용_제거(불용어)'] = df_labor['판례내용_제거(불용어)'].fillna(0)

In [13]:
data_list = df_labor['판례내용_제거(불용어)']

In [15]:
print(data_list.info())
print(data_list.head())

<class 'pandas.core.series.Series'>
RangeIndex: 5016 entries, 0 to 5015
Series name: 판례내용_제거(불용어)
Non-Null Count  Dtype 
--------------  ----- 
5016 non-null   object
dtypes: object(1)
memory usage: 39.3+ KB
None
0    주식회사 ○○ 김소영 피상 중앙 노동 위원장 법무 법인 지향 김진 법무 법인 서희원...
1    피상 조창 ○○○ 주식회사 김원정 부산 고법 창원 비용 부담 한다 한다 판시와 협력...
2    피상 명단 방광 ○○○ 주식회사 김원정 부산 고법 창원 비용 부담 한다 지난 보충 ...
3    피상 법무 법인 금강 ○○○ 주식회사 김원정 주식회사 △△△ 법무 법인 유한 태평양...
4    법무 법인 창조 피상 대한민국 법무 법인 유한 세종 김주안 고법 연차 휴가 수당 주...
Name: 판례내용_제거(불용어), dtype: object


메모리 사용량 찾기

In [17]:
data_list.info(memory_usage="deep")

<class 'pandas.core.series.Series'>
RangeIndex: 5016 entries, 0 to 5015
Series name: 판례내용_제거(불용어)
Non-Null Count  Dtype 
--------------  ----- 
5016 non-null   object
dtypes: object(1)
memory usage: 33.5 MB


<<<<<메모리 줄이는 작업 시작 >>>>>

명사추출

In [15]:
#명사추출이랑 같음


okt = Okt()
for text in data_list:
    nouns = okt.nouns(text)
    docs_list4.append(' '.join(nouns))

n-gram 원본

In [ ]:
### 변경 전 메모리 부족으로 오류 발생 
### 최대 단위 수 지정

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

#1-gram
vect = CountVectorizer() 
dtm = vect.fit_transform(docs_list4).toarray()
vocab = vect.get_feature_names_out()
df_1gram = pd.DataFrame(dtm, columns=vocab)

#2-gram
vect_2gram = CountVectorizer(ngram_range=(2, 2))
dtm_2gram = vect_2gram.fit_transform(docs_list4).toarray()
vocab_2gram = vect_2gram.get_feature_names_out()
df_2gram = pd.DataFrame(dtm_2gram, columns=vocab_2gram)

#3-gram 
vect_3gram = CountVectorizer(ngram_range=(3, 3))
dtm_3gram = vect_3gram.fit_transform(docs_list4).toarray()
vocab_3gram = vect_3gram.get_feature_names_out()
df_3gram = pd.DataFrame(dtm_3gram, columns=vocab_3gram)

print("1-gram")
print(df_1gram)

print("2-gram")
print(df_2gram)

print("3-gram")
print(df_3gram)

* n-gram 변경코드
* chunk size

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

def process_in_chunks(docs_list4, ngram_range=(1, 1), max_features=None, chunk_size=10):
    df_list = []  # 결과를 저장할 리스트
    vect = CountVectorizer(ngram_range=ngram_range, max_features=max_features)
    
    # 데이터셋을 청크로 나누어 처리
    for i in range(0, len(docs_list4), chunk_size):
        chunk = docs_list4[i:i + chunk_size]
        dtm_chunk = vect.fit_transform(chunk)  # 희소 행렬로 유지
        vocab_chunk = vect.get_feature_names_out()
        
        # 희소 행렬을 데이터프레임으로 변환 (toarray()를 사용하지 않고 처리)
        df_chunk = pd.DataFrame.sparse.from_spmatrix(dtm_chunk, columns=vocab_chunk)
        df_list.append(df_chunk)
    
    # 모든 청크를 하나의 데이터프레임으로 결합
    return pd.concat(df_list, ignore_index=True)

# 2-gram 처리 (단어 수 제한, 메모리 문제 방지)
df_2gram = process_in_chunks(docs_list4, ngram_range=(2, 2))

# 결과 출력
print("2-gram")
print(df_2gram)

In [ ]:
# 빈도수 시각화 함수
def plot_top_n(df, title, n=None):
    # 각 n-gram의 빈도 계산
    freq = df.sum().sort_values(ascending=False).head(n)
    
    # 시각화
    plt.figure(figsize=(10, 6))
    plt.barh(freq.index, freq.values, color='skyblue')
    plt.xlabel('빈도')
    plt.title(title)
    plt.gca().invert_yaxis()  # 빈도순으로
    plt.show()

# 상위 20개 빈도 시각화
plot_top_n(df_1gram, '1-gram 빈도',20)
plot_top_n(df_2gram, '2-gram 빈도',20)
plot_top_n(df_3gram, '3-gram 빈도',20)

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import time
import os

# 불용어 설정
stop_words = ['이', '그', '저']

# Okt 객체 생성
okt = Okt()

# 데이터 청크 분할
def split_data(data, chunk_size=250):
    return np.array_split(data, len(data) // chunk_size)

# 명사 필터링 및 n-gram 생성 함수
def process_ngrams(chunk):
    processed_texts = []
    for text in chunk['text_column']:
        nouns = okt.nouns(text)
        filtered_nouns = ' '.join([noun for noun in nouns if noun not in stop_words])
        processed_texts.append(filtered_nouns)
    chunk['processed_text'] = processed_texts
    return chunk

# 데이터 로드 및 분할
data = pd.DataFrame({'text_column': ["여기에 분석할 텍스트를 넣으세요."] * 5016})
data_chunks = split_data(data)

# 병렬 처리 시작
start_time = time.time()
with ProcessPoolExecutor() as executor:
    results = list(executor.map(process_ngrams, data_chunks))

# 처리 결과 병합
processed_data = pd.concat(results, ignore_index=True)

# 3-gram 벡터화
vectorizer = CountVectorizer(ngram_range=(3, 3), max_df=0.9)
X = vectorizer.fit_transform(processed_data['processed_text'])

# 완료 시간 출력
total_time = time.time() - start_time
print(f"전체 처리 완료, 총 경과 시간: {total_time:.2f}초")

# n-gram 목록 저장
ngram_features = vectorizer.get_feature_names_out()
pd.DataFrame(ngram_features, columns=['3-gram']).to_csv("ngram_features.csv", index=False)
